## Standard Method Notebook

### Overview: 

##### Import necessary functions

These functions are stored in src/utils and contain intuitive folders hosting functions. The working directory should be set to the VIEWS_FAO_INDEX folder to enable access to retrieve these .py files.

In [1]:
import os
import sys

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print("The current Working Directory is:", current_directory)

# Get the path to the base directory (VIEWS_FAO_index)
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print(f'The base directory will be set to: {base_dir}')

# Add the base directory to sys.path
sys.path.insert(0, base_dir)

The current Working Directory is: /Users/gbenz/Documents/VIEWS_FAO_index/notebooks/methods
The base directory will be set to: /Users/gbenz/Documents/VIEWS_FAO_index


In [2]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

#Functions necessary for all methods:
from src.utils.universal_functions.setup.generate_base_file import give_primary_frame
from src.utils.universal_functions.setup.associate_country_id import associate_country_years, pull_from_c_y_dictionary
from src.utils.universal_functions.setup.build_directory import ensure_directory_exists


#Functions to generate the insurance table requested by FAO partners:
from src.utils.universal_functions.FAO_table_formatting.calculate_percentiles import format_stats, clean_percentile_table
from src.utils.universal_functions.FAO_table_formatting.generate_output_tables import insurance_table, annual_summary_table

#Functions bespoke to standard method:
from src.utils.functions_for_method_standard.standard_per_capita_fatalities import native_per_capita_fatalities

#Function for Event-Year Return Period process:
from src.utils.functions_for_single_cell_return_period.cell_return_period import calculate_cumulative_distribution, calculate_probabilities, calculate_expected_time_periods, calculate_expected_voxels, compare_empirical_vs_expected
from src.utils.functions_for_single_cell_return_period.Ei_insurance_table_setup import insurance_table_for_E_i

### First, we define our primary dataframe. 

The variable 'data' is developed from two querysets 1. Fatalities_fao_pgm and 2. cm_properties which are required in order to capture fatality attributes and unique country identifiers, including start and end years for boundary changes.

Year parameters can be adapted to incorporate future annual releases of Uppsala Conflict Data Program (UCDP) Geospatial Events Database (GED) data or constrain the temporal window for unique applications. The recommendation is to capture data from the full extent, which at the time of this release is 1989 - 2023.

In [3]:
data = give_primary_frame('Fatalities_fao_pgm', 'cm_properties', 1988, 2023)

100%|██████████| 40.0M/40.0M [00:01<00:00, 26.0MB/s]


Queryset Fatalities_fao_pgm read successfully 
Queryset cm_properties read successfully 


/Users/gbenz/miniforge3/envs/viewser/lib/python3.11/site-packages/ingester3/scratch.py:31: UserWarning: No database connection! Will try to use cache for read-only ops as much as I can
  warnings.warn("No database connection! Will try to use cache for read-only ops as much as I can")


['month_id', 'pg_id', 'country_name', 'C_start_year', 'C_end_year', 'pop_gpw_sum', 'ged_sb', 'ged_ns', 'ged_os', 'year', 'fatalities_sum', 'country_id']


#### Make a copy of the variable data to keep a clean version accesible.

In [4]:
data_working_copy = data

#### Specifiy countries to investigate

In [9]:
countries = ['South Sudan']

#### Process for Country Year Return Period:

In [10]:
for country in countries:
    print('working on: '+ country)

#   First:
#   1. Aggregate the standard PRIO-Grid scale to a courser resolution
    country_and_year_dictionary = associate_country_years(data_working_copy, country)
    print('printing he country and year dictionary:')
    print(country_and_year_dictionary)
#-----------------------------------------------------------------------------------------------------------------------------------
#   Second:
#       1. Subset designated country in list. This requires examining country_id information and corresponding start and end years. 
#   Some countries contain more than one country_id. The functions employed in this section identify the most recent country range and 
#   subset the neccesary temporal ranges.
#-----------------------------------------------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------------------------------------
    cid_int = pull_from_c_y_dictionary(country_and_year_dictionary)
    subset_to_country = data_working_copy[data_working_copy['country_id'] == cid_int]
    conflict_profile = {col: subset_to_country[col].sum() for col in ['ged_sb', 'ged_ns', 'ged_os', 'fatalities_sum']}

    df_annual = native_per_capita_fatalities(subset_to_country)
    df_annual['percapita_100k'] = df_annual['percapita_100k'].round(1)

    percentile_df = format_stats(df_annual)
    filtered_x = clean_percentile_table(percentile_df)
    insurance_table_df = insurance_table(filtered_x, df_annual, ['90','95','98','99','100']) #uses the default attribute = percapita_100k and appened_1_value = yes
    annual_summary = annual_summary_table(df_annual, 'percapita_100k')
#-----------------------------------------------------------------------------------------------------------------------------------
#----- SET DIRECTORIES 
#-----------------------------------------------------------------------------------------------------------------------------------
#----- THIS SETS A DIRECTORY THAT IS UNIQUE UNIQUE STANDARD METHOD ----
#-----------------------------------------------------------------------------------------------------------------------------------
#----- <<< working just with the 'Cell Year' Return Period Process >>>-
    output_path = base_dir + '/notebooks/methods/Country_Results/' + country + '/Standard/Country Year/FAO tables/'
    ensure_directory_exists(output_path)
#-----------------------------------------------------------------------------------------------------------------------------------
    annual_summary_file_path = output_path + country + ' annual summary.csv'
    print(f'saving annual_summary table to: {annual_summary_file_path}')
#-----------------------------------------------------------------------------------------------------------------------------------
    insurance_table_file_path = output_path + country + ' insurance table.csv'
    print(f'saving insurance table to: {insurance_table_file_path}')
#-----------------------------------------------------------------------------------------------------------------------------------
    main_dataframe_file_path = output_path + country + ' main dataframe.csv'
    print(f'saving main dataframe table to: {main_dataframe_file_path}')
#-----------------------------------------------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------------------------------------
#----- NOW WE WRITE TO THE FOLDERS. -----------------------------------
    annual_summary.to_csv(annual_summary_file_path)
    insurance_table_df.to_csv(insurance_table_file_path)
    df_annual.to_csv(main_dataframe_file_path)

working on: South Sudan
   c_id country_name  C_start_year  C_end_year      0
0   246  South Sudan          2011        2050  28096
the length of country_ids for the selected country is: 1
printing he country and year dictionary:
{246: (2011, 2050)}
Index where percapita_100k equals 1: 87.4: 1.5332000000000334
Directory '/Users/gbenz/Documents/VIEWS_FAO_index/notebooks/methods/Country_Results/South Sudan/Standard/Country Year/FAO tables' already exists.
saving annual_summary table to: /Users/gbenz/Documents/VIEWS_FAO_index/notebooks/methods/Country_Results/South Sudan/Standard/Country Year/FAO tables/South Sudan annual summary.csv
saving insurance table to: /Users/gbenz/Documents/VIEWS_FAO_index/notebooks/methods/Country_Results/South Sudan/Standard/Country Year/FAO tables/South Sudan insurance table.csv
saving main dataframe table to: /Users/gbenz/Documents/VIEWS_FAO_index/notebooks/methods/Country_Results/South Sudan/Standard/Country Year/FAO tables/South Sudan main dataframe.csv


In [ ]:
df_annual_cleaned, insurance_table_df, annual_summary = standard_Country_Year_files(data_working_copy, 'Ethiopia')

#### Process for Event Year Return Period:

In [ ]:
for country in countries:

    output_path = base_dir + '/notebooks/methods/Country_Results/' + country + '/Standard/Event Year/FAO tables/'
    main_dataframe_file_path = output_path + country + ' main dataframe.csv'
    print(main_dataframe_file_path)
    aggregation_annual__country = pd.read_csv(main_dataframe_file_path)    

    #This cell is exclusively working with E_i values (Return Period by Cell)
    aggregation_annual__country_renamed = aggregation_annual__country.rename(columns={'GIS__Index': 'priogrid_gid'})
    cumulative_distribution = calculate_cumulative_distribution(aggregation_annual__country_renamed, 'percapita_100k')

    #calculate_probabilities(cumulative_distribution, data, id_column='percapita_100k'):
    probabilities = calculate_probabilities(cumulative_distribution, aggregation_annual__country_renamed, 'pg_id')
    #print(probabilities)
    probabilities['E_i'] = calculate_expected_time_periods(probabilities['P_i'])
    # Calculate E_i^voxels
    probabilities['E_i_voxels'] = calculate_expected_voxels(probabilities['p_i'])
    probabilities_renamed = probabilities.rename(columns={'value': 'percapita_100k'})
    print(probabilities_renamed.head(100))

    probabilities_with_empirical = compare_empirical_vs_expected(aggregation_annual__country_renamed, probabilities_renamed)
    probabilities_with_empirical_sorted = probabilities_with_empirical.sort_values(by='E_i_value')
    subset_E_i = probabilities_with_empirical_sorted[probabilities_with_empirical_sorted['E_i_value'] >= 4.0]
    insurance_from_E_i = insurance_table_for_E_i([5,10,20,30], subset_E_i, aggregation_annual__country_renamed)
    insurance_from_E_i = insurance_from_E_i.round({
        'Closest E_i': 1,
        'percapita_100k': 1,
    })
#-----------------------------------------------------------------------------------------------------------------------------------
#----- SET DIRECTORIES 
#-----------------------------------------------------------------------------------------------------------------------------------
#----- THIS SETS A DIRECTORY THAT IS UNIQUE FOR AGGREGATION METHOD ----
#-----------------------------------------------------------------------------------------------------------------------------------
#----- <<< working just with the 'Cell Year' Return Period Process >>>-
    output_path = base_dir + '/notebooks/methods/Country_Results/' + country + '/Standard/Event Year/FAO tables/'
    ensure_directory_exists(output_path)
#-----------------------------------------------------------------------------------------------------------------------------------
    E_i_insurance_table_file_path = output_path + country + ' Event year insurance table.csv'
    print(f'saving insurance table to: {E_i_insurance_table_file_path}')
#-----------------------------------------------------------------------------------------------------------------------------------
    event_year_probabilities_file_path = output_path + 'Event year probabilities.csv'
    print(f'saving main dataframe table to: {event_year_probabilities_file_path}')
#-----------------------------------------------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------------------------------------
#----- NOW WE WRITE TO THE FOLDERS. -----------------------------------
    insurance_from_E_i.to_csv(E_i_insurance_table_file_path)
    probabilities_with_empirical_sorted.to_csv(event_year_probabilities_file_path)
